> <p><small><small>This Notebook is made available subject to the licence and terms set out in the <a href = "http://www.github.com/google-deepmind/ai-foundations">AI Research Foundations Github README file</a>.

<img src="https://storage.googleapis.com/dm-educational/assets/ai_foundations/GDM-Labs-banner-image-C5-white-bg.png">

# Lab: Implement LoRA for Parameter-Efficient Fine-Tuning

<a href='https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_5/gdm_lab_5_5_implement_lora_for_parameter_efficient_fine_tuning.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

40 minutes

Apply LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning.


## Overview

In the last lab, you observed that it is difficult or impossible to perform full-parameter fine-tuning of a large language model without access to special resources. The solution to this problem is to use fewer parameters following a strategy of **parameter-efficient fine-tuning**.

As you have encountered in the previous article, **Low-Rank Adaptation (LoRA)** is a widely used strategy to fine-tune a transformer without training all of its parameters [1]. LoRA is based on the idea that many matrices can be approximated by multiplying two considerably smaller matrices. The weight updates during fine-tuning then affect only these smaller matrices, which makes LoRA a lot more efficient than full-parameter fine-tuning. Furthermore, in practice, LoRA achieves about the same performance as full-parameter fine-tuning, so despite the lower memory requirement, there is little to no degradation in performance.

In this lab, you will implement LoRA for your small language model.


### What you will learn:

By the end of this lab, you will be able to:

* Explain the mathematical principles underlying the LoRA method.
* Implement LoRA.
* Quantify the memory savings of using LoRA compared with full parameter fine-tuning.


### Tasks

**In this lab, you will**:
* Familiarize yourself with the mathematics behind LoRA.
* Implement your own LoRA layer in Keras.
* Compute how many fewer trainable parameters there are in LoRA compared with full parameter fine-tuning.

Note that you will focus on the implementation of a single layer, which can be done on a **Colab instance with a CPU**.



## How to use Google Colaboratory (Colab)

Google Colaboratory (also known as Google Colab) is a platform that allows you to run Python code in your browser. The code is written in cells that are executed on a remote server.

To run a cell, hover over a cell, and click the `run` button to its left. The run button is the circle with the triangle (▶). Alternatively, you can also click a cell and use the keyboard combination Ctrl+Return (or ⌘+Return if you are using a Mac).

To try this out, run the following cell. This should print today's day of the week below it.

In [ ]:
from datetime import datetime
print(f"Today is {datetime.today():%A}.")

Note that the order in which you run the cells matters. When you are working through a lab, make sure to always run all cells in order. Otherwise, the code might not work. If you take a break while working on a lab, Colab may disconnect you; in that case, you have to execute all cells again before  continuing your work. To make this easier, you can select the cell you are currently working on and then choose __Runtime → Run before__  from the menu above (or use the keyboard combination Ctrl/⌘ + F8). This will re-execute all cells before the current one.

### Using Colab with a GPU

Follow these steps to run the activities in this lab on a GPU:

1.  In the top menu bar, click **Runtime**.
2.  Select **Change runtime type** from the dropdown menu.
3.  In the pop-up window under **Hardware Accelerator**, select **GPU** (usually listed as `T4 GPU`).
4.  Click **Save**.

Your Colab session will now restart with GPU access.

Note that access to GPUs is limited and at times, you may not be able to run this lab on a GPU. All activities will still work but they will run slower and you will have to wait longer for some of the cells to finish running.


## Imports

In this lab, you will use the Keras package for defining your LoRA layer and the Jax package for working with vectors and matrices.

Run the following cell to import the required packages.

In [ ]:
%%capture
# Install the custom package for this course.
!pip install "git+https://github.com/google-deepmind/ai-foundations.git@main"

import os # For setting Keras configuration variables.

# The following two lines provide configuration for Keras.
os.environ["KERAS_BACKEND"] = "jax"

import keras # For building a model.
from keras import layers # As base class for a custom LoRA layer.
keras.utils.set_random_seed(812) # For Keras layers.

import jax # For working with vectors and matrices.
import jax.numpy as jnp # For working with vectors and matrices.

# For checking your solutions.
from ai_foundations.feedback.course_5 import lora as feedback

## Recap

In the lab "Full-Parameter Fine-Tuning", you fine-tuned a small transformer by continuing to update all of its parameters. You observed that this approach did not work for large models. When you tried to train all the parameters of Gemma-1B in Colab, it ran out of memory.  Even though you could circumvent this problem by buying or renting machines with more powerful GPUs, this is costly. Further, this issue gets worse when you consider bigger models. For example, Gemma-1B with its 1.3 billion parameters is relatively small for a large language model and there exist implementations of Gemma with considerably more parameters, such as Gemma-27B [2]. Full-parameter fine-tuning such a model on a single GPU would be impossible even with the most powerful GPUs available today.

As mentioned before, LoRA does not directly update the weights of a matrix $W$. Instead, it keeps the pre-trained weights frozen in $W_0$, and encodes any updates to $W$ as the sum of the pre-trained weights $W_0$ and the weight updates $\Delta W$:

$W = W_0 + \Delta W$

The key property that makes LoRA so efficient is to approximate the weight updates during fine-tuning $\Delta W$ with two much smaller matrices $A$ and $B$, as shown in the figure below:

$\Delta W = BA$

The following section helps you to build your intuition of how LoRA works. For simplicity, consider the weight matrix of one layer of a multi-layer perceptron. Note, this general concept could be applied to any weight matrix of any type of layer.



<img src="https://storage.googleapis.com/dm-educational/assets/ai_foundations/lora.png">

## Intuition behind matrix decomposition with lower ranks

A dense weight matrix, that is a matrix where every entry is assigned a value, explicitly defines the weights for every input and every neuron in a layer. For example, the number in row 5 and column 3 is the weight that goes from the fifth input unit to compute the output of the third neuron. The number of rows is the number of inputs, the number of columns is the number of neurons.

Imagine that you have designed a weight matrix manually based on the data that you observe. You would not, and probably could not, observe each connection individually. You would rather make some general observations, for example:
*   Inputs 1 and 2 do not matter at all.
*   Inputs 3 and 6 reverse the sign.
*   For any input, outputs 4 and 7 have five times the magnitude of the other outputs.

You can encode these observations about the outputs in a matrix with one row, matrix $A$, and you can encode the observations about the inputs in one column, matrix $B$. The product of $B$ and $A$ will be a full weight matrix that specifies each connection between each input and each neuron's output.

The following code implements the above observations in matrices $A$ and $B$, and multiplies them.

Run the following code and carefully inspect the output of the matrix $\Delta W$ in relation to the matrices $A$ and $B$.



In [ ]:
A = jnp.array([[1, 1, 1, 5, 1, 1, 5, 1]]) # Matrix with one row.
B = jnp.array([[0, 0,-1, 1, 1,-1, 1, 1]])
B = jnp.transpose(B) # One column.

# Compute Δ𝑊 by performing the matrix multiplication B x A.
delta_W = jnp.matmul(B,A)
print(delta_W)

As this matrix shows, the resulting matrix product leads to several regularities. This is no coincidence. The zeros in B lead to the rows of zeros in $\Delta W$ and the elements that are negative in B lead to the negative rows in $\Delta W$. The fives in A lead to a five times higher weight in columns 4 and 7.

You can investigate these regularities further with some example inputs:


In [ ]:
# All ones input.
x = jnp.array([1, 1, 1, 1, 1, 1, 1, 1])
y = jnp.matmul(x,delta_W)

print(f"Output for ones only: {y}")

# High values in inputs that do not matter.
x = jnp.array([100, 100, 1, 1, 1, 1, 1, 1])
y = jnp.matmul(x,delta_W)

print(f"Output for high 1st and 2nd input: {y}")

# Higher values (7) in the two inputs that reverse the sign.
x = jnp.array([1, 1, 7, 1, 1, 7, 1, 1])
y = jnp.matmul(x,delta_W)

print(f"Output with input that reverses the sign: {y}")

In [ ]:
# All ones input.
x = jnp.array([1, 1, 1, 1, 1, 1, 1, 1])
y = jnp.matmul(x, delta_W)

print(f"Output for ones only: {y}")

# High values in inputs that do not matter.
x = jnp.array([100, 100, 1, 1, 1, 1, 1, 1])
y = jnp.matmul(x, delta_W)

print(f"Output for high 1st and 2nd input: {y}")

# Higher values (7) in the two inputs that reverse the sign.
x = jnp.array([1, 1, 7, 1, 1, 7, 1, 1])
y = jnp.matmul(x, delta_W)

print(f"Output with input that reverses the sign: {y}")

If you add an additional row to $A$, and an additional column to $B$, you can add an additional observation, such as:
* Input 3 has no effect on output 7.

In [ ]:
A = jnp.array(
    [
        [1, 1, 1, 5, 1, 1, 5, 1],
        [0, 0, 0, 0, 0, 0, 5, 0]
    ]
)
B = jnp.array(
    [
        [0, 0,-1, 1, 1,-1, 1, 1],
        [0, 0, 1, 0, 0, 0, 0, 0 ]
    ]
)
B = jnp.transpose(B)

delta_W = jnp.matmul(B, A)
print(delta_W)

As you can see in the output, the value in the seventh column of the third row of the resulting matrix ends up being 0, encoding the observation above. The encoding of this observation in $A$ and $B$ is a less direct than in the cases above (the 5 in the second row of $A$ and the 1 in the second column of $B$ cancel out the -5 in $\delta W$ above) but what this example demonstrates is that adding more columns to $B$ and more rows to $A$ allows you to encode additional observations.

Weight matrices in a transformer have hundreds or thousands of inputs and outputs. When you decompose $\Delta W$ with two matrices of rank 8, $A$ and $B$ are much smaller than $\Delta W$ but they can still each encode several such general observations, which is often sufficient when performing fine-tuning.

In this context, also note that you are using $\Delta W$ to store only the updates relative to the full pre-trained weight matrix $W_0$. The low-rank approximation of LoRA generally gives the model enough flexibility to encode the details needed to learn the intricacies of a specific task, while the full weight matrix $W_0$ can still encode large quantities of information acquired during pre-training.

## Coding Activity 1: Implement LoRA

You will now implement this method as part of a layer in a multi-layer-perceptron. Recall from the course 03 Design and Train Neural Networks that you can define a hidden layer in Keras using a `Dense` layer.

As a first step, study the cell below that implements a simplified version of the `Dense` layer in Keras. You do not need to run this code but consider the following questions:

- What does the `__init__` function do?.
- How are weights added with `add_weight()`?
- That the `call()` method computes the weighted sum for each neuron and adds the bias term `b`.




In [ ]:
class SimplifiedDense(layers.Layer):
    """A simplified implementation of a dense layer.

    Args:
      output_dim: The number of neurons in the layer.
      input_dim: The number of inputs to each neuron.
    """
    def __init__(self, output_dim: int = 32, input_dim: int = 32):
        """ Initializes a Dense layer."""
        super().__init__() # Call the constructor of the base class.

        self.w = self.add_weight( # w is a weight matrix.
            shape=(input_dim, output_dim), # Shape of w.
            initializer="random_normal", # Intialize the weights randomly.
            trainable=True # w shall be updated during training.
        )
        # Add b as bias weights, which is a vector.
        self.b = self.add_weight(
            shape=(output_dim,),
            initializer="zeros",
            trainable=True
        )

    def call(self, inputs: jax.Array):
        """Multiplies inputs with weight matrix and adds bias.

        Args:
          inputs: The input to the layer.

        Returns:
          The output of the layer.
        """

        # Compute y = XW + b
        return jnp.matmul(inputs, self.w) + self.b

You will now extend this class so that it implements LoRA. The goal here is to write a class that can act as a replacement of a regular `Dense` layer when fine-tuning a model.

This layer therefore needs to:
1. Store the original weight matrix $W_0$.
2. Initialize the weight matrices $A$ and $B$ using rank $r$.
3. In the forward pass (implemented in `call`):
    *  Compute $\Delta W$ using $A$ and $B$.
    *  Compute the new weight matrix $W = W_0 + \Delta W$.

------
> **💻 Your task:**
>
> Your task is to implement the forward function (the `call` method).
>
> The following cell already defines the constructor of the LoRA layer that both stores the original weight matrix $W_0$ and initializes the LoRA weights $A$ and $B$ in the constructor `__init__`.
>
> Walk through the constructor below and then implement the `call` method as outlined in Step 3 above.
>
------



In [ ]:
class LoraDense(layers.Layer):
  """An implementation of a dense layer with LoRA.

  Args:
    pretrained_layer: The original pre-trained dense layer.
    rank: The rank of the LoRA matrices.

  Attributes:
    num_units: The number of units in the dense layer.
    rank: The rank of the LoRA matrices.
    A: The LoRA matrix A.
    B: The LoRA matrix B.
    W0: The pre-trained weights.
    bias: The pre-trained bias parameters.
  """
  def __init__(self , pretrained_layer: layers.Dense, rank: int = 8, **kwargs):
    """Initializes a LoRA layer.

    Args:
      pretrained_layer: The original pre-trained dense layer.
      rank: The rank of the LoRA matrices.
    """

    super().__init__(**kwargs)  # Call the constructor of the base class.

    # Shape of the weight matrix W of the pre-trained layer.
    pretrained_layer_shape = pretrained_layer.weights[0].shape
    # Shape of the bias in the pre-trained layer.
    bias_shape = pretrained_layer.weights[1].shape
    # Number of units = number of columns of W of the pre-trained layer.
    self.num_units = pretrained_layer_shape[1]
    # Rank of the Lora matrices.
    self.rank = rank

    # LoRA matrix A, shape rank x num_units.
    # Note that in the original LoRA paper, this matrix is initialized
    # with random numbers. This implementation also uses this convention.
    self.A = self.add_weight(
        shape=(rank, self.num_units),
        initializer=keras.initializers.RandomNormal(stddev=0.01),
        trainable=True, # These parameters should be updated during fine-tuning.
    )
    # LoRA matrix B, shape number of inputs x r.
    # Note that in the original LoRA paper, this matrix is initialized
    # with all values being set to 0. This implementation also uses this
    # convention.
    self.B = self.add_weight(
        shape=(pretrained_layer_shape[0], rank),
        initializer=keras.initializers.Zeros(),
        trainable=True, # These parameters should be updated during fine-tuning.
    )
    # Pre-trained weights W0, copied from pre-trained layer.
    self.W0 = self.add_weight(
        shape=(pretrained_layer_shape),
        initializer=keras.initializers.Constant(pretrained_layer.weights[0]),
        trainable=False, # These parameters are frozen during fine-tuning.
    )
    # Bias weights bias, copied from pre-trained layer.
    self.bias = self.add_weight(
        shape=(bias_shape),
        initializer=keras.initializers.Constant(pretrained_layer.weights[1]),
        trainable=True, # These parameters should be updated during fine-tuning.
    )

  def call(self, x: jax.Array):
    """
      Forward pass through the layer.

      Args:
        x: The input to the layer.

      Returns:
        The output of the layer.
    """

    # Compute ΔW = BA.
    delta_W = # Add your code here.

    # Compute W = W0 + ΔW.
    W = # Add your code here.

    # Compute layer output y = xW + b.
    y = # Add your code here.

    return y

To test the implemenation of your LoRA layer, you can implement a small one-layer MLP and then replace its dense layer with a LoRA layer.

The following cell first defines a model that takes the role of a pre-trained model (`pretrained_model`) and then initializes another model with a LoRA dense layer (`lora_model`).

Run the following cell to make sure your code executes correctly. It should print the shape of the output of both models. If you implemented the forward function correctly, it should execute without any errors and the two output shapes should be identical.

In [ ]:
# Number of units for inputs and outputs.
# In a transformer, this is typically the embedding space, size of the
# attention mechanism, and the output of a transformer block.
num_units = 512
rank = 8

# Define a model with a single dense layer in pretrained_model.
x = layers.Input(shape=(num_units,))
y = layers.Dense(units=num_units, name="dense_layer_1")(x)
pretrained_model = keras.Model(inputs=x, outputs=y)

# Take the dense layer to convert to a LoRA layer.
pretrained_layer = pretrained_model.get_layer("dense_layer_1")

# Define a model with a single LoRA layer.
x = layers.Input(shape=(num_units,))
y = LoraDense(
    pretrained_layer=pretrained_layer,
    rank=rank,
    name="lora_dense_layer_1"
)(x)
lora_model = keras.Model(inputs=x, outputs=y)

# Pass an example input through both the original model and the LoRA model.
input = jnp.ones((1, num_units))
output_original = pretrained_model(input)
output_lora = lora_model(input)

# Print the shapes of the output from both models.
print(f"Shape of original model output: {output_original.shape}")
print(f"Shape of LoRA model output: {output_lora.shape}")

## Coding Activity 2: Compare number of trainable parameters

As a final step, investigate how much the LoRA layer reduces the number of trainable parameters in a model.

------
> 💻 **Your task:**
>
> Call the [`summary()`](https://keras.io/api/models/model/#summary-method) method for the pre-trained model (`pretrained_model`) and the LoRA model (`lora_model`) above.
>
> Consider the following questions:
>*   How many weights do each of the two layers have?
>*   How many of those weights are trained?
>
------

In [ ]:
# Add your code here.

### What did you observe?

When you consider the number of total parameters, you should have observed that this number was greater for the LoRA model than for the original model. This is because the LoRA model adds the matrices $A$ and $B$ to the model. However, given that both matrices are relatively small compared to the full weight matrix $W_0$, the increase in parameters is also relatively small.

On the other hand, when you consider the number of *trainable* parameters, then this number is a lot smaller for the LoRA model than for the original model. This is because the LoRA model is not updating $W_0$, which defines the majority of the model parameters.


### Activity 3: Compute the proportion of trainable parameters

In the previous activity, you observed that the number of trainable parameters is smaller for the LoRA model than for the original model. In this activity,
consider how much LoRA reduces the number of trainable parameters.


------
> 💻 **Your task:**
>
> Compute the proportion of trainable parameters in a dense layer with LoRA compared to a regular dense layer.
>
> Assume that the input and output size is 512.
>
------

In [ ]:
# @title Enter your response (a number between 0.0 and 1.0) here
answer = 0.0331 #@param {type: "number"}

In [ ]:
# @title Run this cell to check your answer

feedback.check_loralab_answer(answer, num_units=512, rank=8)

## LoRA in a transformer

The goal of LoRA is to reduce the number of parameters when fine-tuning a model. Since both the attention mechanism and the multi-layer perceptron in every transformer block contain dense layers, it is possible to apply LoRA to some or all of these dense layers, resulting in a much lower number of layers that need to be trained [1].

LoRA has become an important tool for fine-tuning neural networks and in fact, the [`layers.Dense`](https://keras.io/api/layers/core_layers/dense/) class in Keras already implements LoRA out of the box, so generally you do not have to implement this from scratch.

To illustrate how you would use this in a real model, consider the transformer model from the course 04 Discover the Transformer Architecture. If you wanted to apply LoRA to this model, you could simply add the parameters `lora_rank` to all dense layers in the model. During pre-training, you would set this parameter to `None`, which disables the application of LoRA. During fine-tuning, you could then set it to a reasonable rank, such as $r=8$ to reduce the number of parameters that need to be updated.

## Summary

In this lab, you implemented LoRA for a dense layer. You learned how $A$ and $B$ affect the rows and columns of $\Delta W$.

In the next lab, you will fine-tune Gemma with LoRA and observe how this makes it possible to fine-tune a much larger model on a GPU such as the T4 GPU available through Colab.

## Solutions

The following cells provide reference solutions to the coding activities in this notebook. If you really get stuck after trying to solve the activities yourself, you may want to consult these solutions.

It is recommended that you *only* look at the solutions after you have tried to solve the activities *multiple times*. The best way to learn challenging concepts in computer science and artificial intelligence is to debug your code piece-by-piece until it works, rather than copying existing solutions.

If you feel stuck, you may want to first try to debug your code. For example, by adding additional print statements to see what your code is doing at every step. This will provide you with a much deeper understanding of the code and the materials. It will also provide you with practice on how to solve challenging coding problems beyond this course.

To view the solutions for an activity, click on the arrow to the left of the activity name. If you consult the solutions, do not copy and paste them into the cells above. Instead, look at them, and type them manually into the cell. This will help you understand where you went wrong.



### Coding Activity 1

In [ ]:
# Complete implementation of forward pass.
def call(self, x: jax.Array) -> jax.Array:
    """
      Forward pass through the layer.

      Args:
        x: The input to the layer.

      Returns:
        The output of the layer.
    """

    # Compute ΔW = BA.
    delta_W = jnp.matmul(self.B, self.A)

    # Compute W = W0 + ΔW.
    W = self.W0 + delta_W

    # Compute layer output y = xW + b.
    y = jnp.matmul(x, W) + self.bias

    return y

### Coding Activity 2




In [ ]:
# Add this code to the cell above.
pretrained_model.summary()
lora_model.summary()

### Activity 3

In the scenario stated in the question, the full weight matrix has 512 x 512 = 262,144 parameters. These are trained in full-parameter tuning and fixed in $W_0$ when using LoRA.

Matrix $A$ has 8 x 512 = 4,096 parameters. Matrix $B$ also has 4,096 parameters. These are trained with LoRA.

Thus, the proportion is (4,096 + 4,096) / 262,144 = 0.03125, or roughly 3.1%.
If you consider bias weights, the calculation is (4,096 + 4,096 + 512) / (262,144 + 512) ≈ 0.03314.

## References

[1] Edward Hu, Yelong Shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, and Weizhu Chen. 2022. LoRA: Low-rank adaptation of large language models. In International Conference on Learning Representations (ICLR 2022). https://openreview.net/pdf?id=nZeVKeeFYf9

[2] Google AI. 2025. Gemma 3 model overview. https://ai.google.dev/gemma/docs/core

[3] François Chollet. 2023. Making new layers and models via subclassing. Keras. https://keras.io/guides/making_new_layers_and_models_via_subclassing/
